# Import Libraries

In [1]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from text_to_sql import (
    TextToSQL,
    Config,
    LLMConfig,
    SLConfig,
    ContextConfig,
    QueryConfig,
)
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
import os

# Constants

In [3]:
MAX_RETRIES = 5
RETRY_DELAY = 2
DATABASE = "academic"
MODEL = "gpt-4.1-mini"
PROVIDER = "openai"

# Load Environment

In [4]:
load_dotenv()

True

# Set Timestamp Experiment

In [5]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_dir = f"../files/experiment_result/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Config

In [6]:
db_key = DATABASE.upper().replace("-", "_")
provider_key = PROVIDER.upper().replace("-", "_")

config = Config(
    max_retry_attempt=5,
    rewriter_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    query_generator_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    schema_linker_config=SLConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
        schema_path=f"../files/schema/{DATABASE}.txt",
        metadata_path=f"../files/metadata/{DATABASE}.json",
    ),
    retrieve_context_config=ContextConfig(data_path=f"../files/dataset/dataset_{DATABASE}.csv"),
    query_executor_config = QueryConfig(
        host=os.getenv(f"DB_HOST_{db_key}"),
        database=os.getenv(f"DB_DATABASE_{db_key}"),
        user=os.getenv(f"DB_USER_{db_key}"),
        password=os.getenv(f"DB_PASSWORD_{db_key}"),
        port=os.getenv(f"DB_PORT_{db_key}"),
    ),
)

# Model

In [7]:
text_to_sql_model = TextToSQL(config=config)

Initializing API client for openai using model gpt-4.1-mini.
Initializing API client for openai using model gpt-4.1-mini.
Initializing API client for openai using model gpt-4.1-mini.


c:\Users\michj\AppData\Local\Programs\Python\Python39\lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


# Import Dataset

In [8]:
dataset = pd.read_csv(f"../files/dataset/dataset_{DATABASE}.csv")

In [9]:
dataset

,Question,Answer,Summary,Alternative Prompt 1 (English),Alternative Prompt 2 (Bahasa Indonesia),Expected Result
0,Which faculty has the most research projects? ...,"SELECT f.nama, COUNT(p.id_proyek) AS total_pro...",This SQL query identifies the faculty with the...,Can you tell me which faculty did the most res...,Fakultas mana yang paling banyak bikin proyek ...,"['nama', 'total_proyek']"
1,Which lecturer supervises the most students? I...,"SELECT d.nama, COUNT(m.id_mahasiswa) AS total_...",This SQL query identifies the lecturer supervi...,I’m curious which lecturer guides the most stu...,Dosen mana yang ngebimbing mahasiswa paling ba...,"['nama', 'total_mahasiswa_dibimbing']"
2,Which student has attended the most sessions? ...,"SELECT m.nama, COUNT(k.id_kehadiran) AS total_...",This SQL query identifies the student with the...,Which student joined the most sessions? I want...,Mahasiswa mana yang paling rajin ikut sesi kel...,"['nama', 'total_kehadiran']"
3,What are the top 5 courses with the highest nu...,"SELECT mk.judul, COUNT(e.id_mahasiswa) AS tota...",This SQL query identifies the top 5 most popul...,What are the top 5 most popular courses? Show ...,Top 5 matkul yang paling banyak diambil siapa ...,"['judul', 'total_mahasiswa']"
4,Which faculty uses the most facilities? If the...,"SELECT f.nama, COUNT(ff.id_fasilitas) AS total...",This SQL query identifies the faculty with the...,Which faculty uses the most facilities? I need...,Fakultas mana yang paling banyak pake fasilita...,"['nama', 'total_fasilitas']"
5,Which course has the highest average exam scor...,"SELECT mk.judul, AVG(h.nilai_diperoleh) AS rat...",This SQL query identifies the course with the ...,Which course has the best average exam score? ...,Matkul mana yang nilai ujiannya paling tinggi ...,"['judul', 'rata_rata_nilai']"
6,Which lecturer has authored the most publicati...,"SELECT d.nama, COUNT(p.id_publikasi) AS total_...",This SQL query identifies the most published f...,Who is the lecturer with the most publications...,Dosen mana yang paling banyak nulis publikasi?...,"['nama', 'total_publikasi']"
7,Which student has participated in the most res...,"SELECT m.nama, COUNT(pp.id_proyek) AS total_pa...",This SQL query identifies the student with the...,Which student has joined the most research pro...,Mahasiswa mana yang paling sering ikut proyek ...,"['nama', 'total_partisipasi']"
8,Which student has borrowed the most books? If ...,"SELECT id_peminjam, COUNT(*) AS total_peminjam...",This SQL query identifies the most active libr...,Who’s borrowed the most books among students? ...,Mahasiswa mana yang paling sering minjem buku?...,"['id_peminjam', 'total_peminjaman']"
9,Which club has the most active members? If the...,"SELECT k.nama, COUNT(kk.id_mahasiswa) AS total...",This SQL query identifies the most popular stu...,Which club has the most members? Show the name...,Klub mana yang anggotanya paling banyak? Kasih...,"['nama', 'total_anggota']"


# Experiment Baseline

In [10]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_baseline(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
LEFT JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.id_fakultas, f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 0.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
Generated SQL Query: SELECT d.nama, COUNT(m.id_mahasisw

In [11]:
df_results_baseline = pd.DataFrame(results_list)
df_results_baseline.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_baseline.csv", index=False)

# Experiment V1

In [12]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v1(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
LEFT JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
[Attempt 1] Failed to generate SQL: API request failed: 429 - {
    "e

In [13]:
df_results_v1 = pd.DataFrame(results_list)
df_results_v1.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v1.csv", index=False)

# Experiment V2

In [14]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v2(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
LEFT JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
[Attempt 1] Failed to generate SQL: API request failed: 429 - {
    "e

In [15]:
df_results_v2 = pd.DataFrame(results_list)
df_results_v2.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v2.csv", index=False)

# Experiment V3

In [16]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v3(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Related Tables: {'ProyekPenelitian', 'Jurusan', 'Fakultas', 'Dosen'}
Generated SQL Query: SELECT f.nama, COUNT(pp.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian pp ON f.id_fakultas = pp.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Related Tables: {'Publikasi', 'Jurusan', 'PenulisPublikasi', 'Fakultas', 'Dosen'}
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their

In [17]:
df_results_v3 = pd.DataFrame(results_list)
df_results_v3.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v3.csv", index=False)

# Eksperimen V4 

In [18]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v4(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
Generated SQL Query: SELECT d.nama, COUNT(m.id_mahasiswa) AS total_mahasisw

In [19]:
df_results_v4 = pd.DataFrame(results_list)
df_results_v4.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v4.csv", index=False)

# Eksperimen V5

In [20]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v5(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Related Tables: {'ProyekPenelitian', 'Jurusan', 'Fakultas', 'Dosen'}
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Related Tables: {'Publikasi', 'Jurusan', 'PenulisPublikasi', 'Fakultas', 'Dosen'}
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
LEFT JOIN ProyekPenelitian p ON p.id_fakultas = f.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show the

In [21]:
df_results_v5 = pd.DataFrame(results_list)
df_results_v5.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v5.csv", index=False)